# Preprocesar encabezados

Este script remplaza dobles espacios, saltos de linea y dobles saltos de linea en los encabezados.

Es necesario crear un archivo llamado `mongo_uri.txt`en el mismo directorio de este script. Este archivo deberá contener la uri de conexión al replica set de MongoDB Atlas

In [1]:
from pymongo import MongoClient, UpdateOne
import re
from pprint import pprint

In [2]:
urifile = open("mongo_uri.txt", 'r', encoding='utf-8')
uri = urifile.read()
client = MongoClient(uri)
db = client.boletines_db
pprint(db.boletines.find_one())

{'_id': ObjectId('5ec3089dd380e649eb619ce9'),
 'boletin': 'Juntos haremos historia\n'
            'BOLETÍN CAMPAÑA-009\n'
            'PIDE LÓPEZ OBRADOR A TRUMP QUE NO MALTRATE A LOS MEXICANOS, '
            '“QUEREMOS RESPETO MUTUO”\n'
            '- NI LAS AMENAZAS, NI LOS MILITARES A LA FRONTERA EVITARAN QUE '
            'MÉXICO QUIERA TENER UNA RELACIÓN DE AMISTAD CON EU: AMLO\n'
            'MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018.- Andrés Manuel López '
            'Obrador manifestó que México y Estados Unidos “somos pueblos que '
            'estamos hermanados, los gobiernos no tienen por qué separar a los '
            'pueblos, nosotros le pedimos al presidente Donald Trump con todo '
            'respeto que no maltrate a los mexicanos, no queremos racismo y '
            'discriminación, queremos respeto mutuo”.\n'
            'Recalcó que nunca se le faltará el respeto al pueblo '
            'estadounidense, pero México y su pueblo no será piñata de ningún '
       

In [3]:
remplazos = [
    # Remplazo dobles lineas por un identificador especial
    (r'\n\n', '<dnl>'),
    # Remplazo lineas simples por espacios
    (r'\n', ' '),
    # Remplazo identificador de linea doble por linea simple
    (r'<dnl>', '\n'),
    # Remplazo caracter &nbsp (\u00A0) por espacio
    (r'\u00A0', ' '),
    # Remplazo tabulador por espacio
    (r'\t', ' '),
    # Remplazo repeticiones de más de dos espacios por espacio simple
    (r' ( )+', ' '),
    # Estandarizo el uso de comillas dobles a “Cita chidorina”
    # Caracteres \u201c “  \u201d ”
    (r'(^|[ \n])\u201d([0-9\wáéíóúÁÉÍÓÚüÜÑñ])', r'\1“\2'),
    (r'(^|[ \n])"([0-9\wáéíóúÁÉÍÓÚüÜÑñ])', r'\1“\2'),
    (r'([0-9\wáéíóúÁÉÍÓÚüÜÑñ])\u201c($|[ ,\.\n])', r'\1”\2'),
    (r'([0-9\wáéíóúÁÉÍÓÚüÜÑñ])"($|[ ,\.\n])', r'\1”\2'),
    # Cambio dos guiones por guión por simple
    (r'--+', r'—'),
    # Cambio estructuras -hola mundo- por —Hola mundo— (caracter \u2014)
    (r'(^|[ \n])-([0-9\wáéíóúÁÉÍÓÚüÜÑñ])', r'\1—\2'),
    (r'([0-9\wáéíóúÁÉÍÓÚüÜÑñ])-($|[ ,\.\n])', r'\1—\2'),
    # Elimino caracter de acento agudo
    (r'\u00B4', ''),
    # Cambio comillas simples \u2018 y \u2019 por '
    (r'[\u2018\u2019]', r"'"),
]

In [5]:
cursor = db.boletines.find({}, {"encabezado":1, "sumarios":1})

In [10]:
update_ops = []
i = 0
for doc in cursor:
    filtro = { '_id' : doc['_id']}
    # Remplazo patrones en encabezado
    for patron, remplazo in remplazos:
        doc['encabezado'] = re.sub(patron, remplazo, doc['encabezado'])
    
    update = { '$set': 
        { 
            'encabezado': doc['encabezado'].strip(),
        }
    }
    
    # Remplazo patrones en sumarios
    try:
        sumarios = []
        for sumario in doc['sumarios']:
            for patron, remplazo in remplazos:
                sumario = re.sub(patron, remplazo, sumario)
            sumarios.append(sumario.strip())
        update['$set']['sumarios'] = sumarios
        if i%30 == 0:
            pprint(filtro['_id'])
            pprint(update['$set']['sumarios'])
            print()
    except KeyError:
        pass
    # Operaciones de actualización
    update_ops.append(UpdateOne(filtro, update))
    i+=1
cursor.rewind()

ObjectId('5ec3089dd380e649eb619ce9')
['En votación efectiva, las encuestas nos dan entre 50 y 52 por ciento, '
 'asegura',
 'Estamos pobremente más de 20 puntos arriba, comenta',
 'Reitera que el repliegue de militares iniciado por Donald Trump es un asunto '
 'político propagandístico con miras a su reelección']

ObjectId('5ec3089dd380e649eb619cef')
['Adelanta AMLO que al triunfar irá a ver su expediente en el CISEN y le '
 'sacará copia, porque es un documento histórico',
 'López Obrador opina que el ex jefe de Gobierno traicionó al pueblo de la '
 'Ciudad de México y por eso se fue a desayunar a otra parte para no verlo',
 '“Uy, uy, uy, uy, hay que miedo, mira cómo estoy temblando” así responde al '
 'mensaje que le envió el candidato independiente Jaime Rodríguez',
 'Andrés Manuel López Obrador lamenta Obrador lamentó que el fallecimiento de '
 'Sergio Pitol']

ObjectId('5ec3089dd380e649eb619cec')
['Pide a los dirigentes de MORENA de la capital para hacerse de la información '
 'so

In [11]:
resultado = db.boletines.bulk_write(update_ops)
print(resultado.bulk_api_result)

{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 347, 'nModified': 57, 'nRemoved': 0, 'upserted': []}


In [13]:
client.close()